In [1]:
import numpy as np
import exafmm.helmholtz as helmholtz

In [2]:
helmholtz.__doc__

"exafmm's submodule for Helmholtz kernel"

### 1. create sources and targets

In [3]:
nsrcs = 100000
ntrgs = 200000

# generate random positions for particles
src_coords = np.random.random((nsrcs, 3))
trg_coords = np.random.random((nsrcs, 3))

# generate random charges for sources
src_charges = np.zeros(nsrcs, dtype=np.complex_)
src_charges.real = np.random.random(nsrcs)
src_charges.imag = np.random.random(nsrcs)

In [4]:
# create a list of source instances
sources = helmholtz.init_sources(src_coords, src_charges)

# create a list of target instances
targets = helmholtz.init_targets(trg_coords)

### 2. create a HelmholtzFmm instance

In [5]:
fmm = helmholtz.HelmholtzFmm(p=10, ncrit=300, depth=4, wavek=10, filename='helmholtz_example_k10.dat')

### 3. setup fmm

Given sources, targets and FMM configurations, `setup()` builds the tree and interaction list and pre-compute invariant matrices.

In [6]:
tree = helmholtz.setup(sources, targets, fmm)

In [7]:
ls *.dat

helmholtz_example_k10.dat  helmholtz_k10.dat  test_file.dat


### 4. evaluate

`evaluate()` triggers the evaluation and returns a $n_{trg} \times 4$ `numpy.ndarray`.
The $i$-th row of `trg_values` starts with the potential value of the $i$-th target, followed by its three gradient values.

In [8]:
trg_values = helmholtz.evaluate(tree, fmm)

In [9]:
trg_values[6]

array([-1348.30005064 +311.77459308j, -1121.90392537  -12.37575463j,
        5461.31955061-4204.03901067j,  -639.45993077  -59.90979191j])

### 5. check accuracy (optional)

`fmm.verify(tree.leafs)` returns L2-norm the relative error of potential and gradient in a list, compared with the values calculated from direct method. 

In [10]:
fmm.verify(tree.leafs)

[4.4101185995483076e-08, 3.9195802256091114e-07]

### 6. update charges of sources and run FMM iteratively

In [11]:
niters = 4

for i in range(niters):
    print('-'*10 + ' iteration {} '.format(i) + '-'*10)  # print divider between iterations
    
    src_charges.real = np.random.random(nsrcs)     # generate new random charges
    src_charges.imag = np.random.random(nsrcs)          
    helmholtz.update_charges(tree, src_charges)      # update charges
    helmholtz.clear_values(tree)                     # clear values
    trg_values = helmholtz.evaluate(tree, fmm)       # evaluate potentials

    print("Error: ", fmm.verify(tree.leafs))       # check accuracy

---------- iteration 0 ----------
Error:  [4.297007196443259e-08, 4.112396937661637e-07]
---------- iteration 1 ----------
Error:  [4.37010633104715e-08, 4.6572604986842655e-07]
---------- iteration 2 ----------
Error:  [4.294779539115475e-08, 4.308971209164518e-07]
---------- iteration 3 ----------
Error:  [4.402664769161639e-08, 5.651934910287226e-07]
